In [10]:
import numpy as np
import torch
import sys
sys.path.append('../')
from voting_games.werewolf_env_v0 import plurality_env, plurality_Phase, plurality_Role
import random
import copy
from typing import Any, Generator, Optional, Tuple
from tqdm import tqdm

In [8]:
env = plurality_env(num_agents=10, werewolves=2)
env.reset()

def random_coordinated_wolf(env):
    villagers_remaining = set(env.world_state["villagers"]) & set(env.world_state['alive'])
    wolves_remaining = set(env.world_state["werewolves"]) & set(env.world_state['alive'])

    target = random.choice(list(villagers_remaining))
    return {wolf: int(target.split("_")[-1]) for wolf in wolves_remaining}

def random_wolfs(env):
    return {wolf: env.action_space(wolf).sample() for
            wolf in set(env.world_state["werewolves"]) & set(env.world_state['alive'])}

def revenge_coordinated_wolf(env, actions = None):
    villagers_remaining = set(env.world_state["villagers"]) & set(env.world_state['alive'])
    wolves_remaining = set(env.world_state["werewolves"]) & set(env.world_state['alive'])

    # who tried to vote out a wolf last time?
    
    target = random.choice(list(villagers_remaining))
    # pick 
    for wolf in wolves_remaining:
        actions[wolf] = [0] * len(env.possible_agents)
        actions[wolf][int(target.split("_")[-1])] = -1
        for curr_wolf in wolves_remaining:
            actions[wolf][int(curr_wolf.split("_")[-1])] = 1
    # for wolf in env.werewolves_remaining:

def random_single_target_villager(env, agent):
    targets = set(env.world_state["alive"]) - set([agent])
    return int(random.choice(list(targets)).split("_")[-1])

# random_coordinated_wolf(env)
def random_agent_action(env, agent):
   return env.action_space(agent).sample()


In [9]:
def play_static_wolf_game(env, wolf_policy, villager_agent, num_times=100) -> tuple(plurality_Role):

    villager_wins = 0
    loop = tqdm(range(num_times))

    for _ in loop:
        next_observations, rewards, terminations, truncations, infos = env.reset()
        while env.agents:
            observations = copy.deepcopy(next_observations)
            actions = {}

            villagers = set(env.agents) & set(env.world_state["villagers"])
            wolves = set(env.agents) & set(env.world_state["werewolves"])

            # villager steps
            if env.world_state["phase"] != plurality_Phase.NIGHT:
                # villagers actions
                for villager in villagers:
                    actions[villager] = villager_agent(env, villager)

            # wolf steps
            actions = actions | wolf_policy(env)
        
            next_observations, rewards, terminations, truncations, infos = env.step(actions)

        winner = env.world_state['winners']
        if winner == plurality_Role.VILLAGER:
            villager_wins += 1

        loop.set_description(f"Villagers won {villager_wins} out of a total of {num_times} games")

env = plurality_env(num_agents=10, werewolves=2)
env.reset()

print("Random Coordinated Wolves")
print("\t vs. Single Target Random Villagers")
play_static_wolf_game(env, random_coordinated_wolf, random_single_target_villager, num_times=1000)
print("\t vs. Random Villagers")
play_static_wolf_game(env, random_coordinated_wolf, random_agent_action, num_times=1000)
print("------------------------------------\n")
print("Random Wolves")
print("\t vs. Single Target Random Villagers")
play_static_wolf_game(env, random_wolfs, random_single_target_villager, num_times=1000)
print("\t vs. Random Villagers")
play_static_wolf_game(env, random_wolfs, random_agent_action, num_times=1000)
print("------------------------------------\n")

Random Coordinated Wolves
	 vs. Single Target Random Villagers


Villagers won 105 out of a total of 1000 games: 100%|██████████| 1000/1000 [00:05<00:00, 176.66it/s]


	 vs. Random Villagers


Villagers won 51 out of a total of 1000 games: 100%|██████████| 1000/1000 [00:04<00:00, 200.31it/s]


------------------------------------

Random Wolves
	 vs. Single Target Random Villagers


Villagers won 689 out of a total of 1000 games: 100%|██████████| 1000/1000 [00:04<00:00, 200.93it/s]


	 vs. Random Villagers


Villagers won 589 out of a total of 1000 games: 100%|██████████| 1000/1000 [00:05<00:00, 175.50it/s]

------------------------------------



In [ ]:
class PluralityAgent(torch.nn.Module):

    def __init__():
        pass

    def forward(self, x):
        h_0
        c_0

In [11]:
class PluralityRolloutBuffer():
    
    def __init__(self, 
                 buffer_size: int, 
                 state_dim: Tuple[int],
                 action_space: gym.Space,
                 gamma: float, 
                 gae_lambda: float,
                 device: str,
                 is_recurrent: bool,
                 recurrent_size: int = None,
                 ):
        self.buffer_size = buffer_size
        self.state_dim = state_dim
        self.action_space = action_space
        self.action_dim = (action_space.n,) if isinstance(action_space, gym.spaces.Discrete) else action_space.shape
        self.gamma = gamma 
        self.gae_lambda = gae_lambda
        self.device = device
        self.is_recurrent = is_recurrent
        if self.is_recurrent:
            self.recurrent_size = recurrent_size
        
        self.reset()

    def reset(self):

        self.rewards = None
        self.actions = None
        self.dones = None
        self.observations = None

        # do we want these for both actor and critic?
        self.hxs = None 
        self.cxs = None

        self.log_probs = None
        self.values = None
        self.advantages = None

        pass

    def add_sample(self, sample: Tuple[Any]) -> bool:
        pass

    def end_trajectory(self, last_value: np.ndarray, last_done: int):


    def sample_batch(self, recurrent_seq_len: int, whole_episodes: bool = False, batch_size: Optional[int] = 256)

SyntaxError: incomplete input (1809460814.py, line 32)

In [ ]:
def fill_buffer(env, wolf_policy, training_agent, num_times=100) -> tuple(plurality_Role):

    villager_wins = 0
    loop = tqdm(range(num_times))

    

    for _ in loop:

        ## Play the game 
        next_observations, rewards, terminations, truncations, infos = env.reset()
        magent_obs = {agent: {'obs': [], 
                              'rewards': [], 
                              'actions': [], 
                              'logprobs': [], 
                              'values': [], 
                              'terms': []} for agent in env.agents if not env.agent_roles[agent]}
        while env.agents:
            observations = copy.deepcopy(next_observations)
            actions = {}

            villagers = set(env.agents) & set(env.world_state["villagers"])
            wolves = set(env.agents) & set(env.world_state["werewolves"])

            # villager steps
            if env.world_state["phase"] != plurality_Phase.NIGHT:
                # villagers actions
                for villager in villagers:
                    obs = torch.Tensor(env.convert_obs(observations[villager]['observation']))
                    ml_action, logprobs, _, value = training_agent.get_action_and_value(obs)

                    actions[villager] = training_agent.get_action_and_value(obs)

                    # can store some stuff 
                    magent_obs[villager]["obs"].append(obs)
                    magent_obs[villager]["actions"].append(ml_action)
                    magent_obs[villager]["logprobs"].append(logprobs)
                    magent_obs[villager]["values"].append(value)

            # wolf steps
            actions = actions | wolf_policy(env)
        
            next_observations, rewards, terminations, truncations, infos = env.step(actions)

            for villager in villagers:
                    if env.history[-1]["phase"] == plurality_Phase.NIGHT:
                        magent_obs[villager]["rewards"][-1] += rewards[villager]
                        magent_obs[villager]["terms"][-1] += terminations[villager]
                    else:
                        magent_obs[villager]["rewards"].append(rewards[villager])
                        magent_obs[villager]["terms"].append(terminations[villager])


        winner = env.world_state['winners']
        if winner == plurality_Role.VILLAGER:
            villager_wins += 1

        ## Update the advantages

        ## Fill bigger buffer, keeping in mind sequence


        loop.set_description(f"Villagers won {villager_wins} out of a total of {num_times} games")

env = plurality_env(num_agents=10, werewolves=2)
env.reset()
